***Follow these instructions to set env variables. Only run once***

Sign up here: https://case.law/user/register/

Click Account, get your API key.

Open terminal

vim ~/.bash_profile

press i to enter insert mode

go to the bottom of the file and add these three lines:

export CASELAW_APIKEY = 'your_api_key'

press escape and the type ':wq!' to save and exit

run source ~/.bash_profile to update your environment

You will need to shut down the jupyter notebook and restart your terminal to see the effects.

In [51]:
import requests
import os
import utils
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/anaconda3/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
#IMPORTANT: need to authenticate every single request, save the api key to bash profile
test_case_response = requests.get(
    'https://api.case.law/v1/cases/4194497/?full_case=true',
    headers={'Authorization': 'Token ' + os.environ['CASELAW_APIKEY']}
)

In [41]:
#utils.get_request_caselaw('https://api.case.law/v1/cases/4194497/?full_case=true').json()

query_results = utils.get_request_caselaw('https://api.case.law/v1/cases/?search="neighbor property"').json()
text_retrieval = list()
for result in query_results['results']:
    caseid = result['id']
    caseresult = utils.get_request_caselaw('https://api.case.law/v1/cases/' + str(caseid) + '/?full_case=true').json()
    opinions = caseresult['casebody']['data']['opinions']
    text = caseresult['name'] + '\n'
    for opinion in opinions:
        text = text + opinion['text'] + '\n'
    text_retrieval.append(text)

In [126]:
# sid = SentimentIntensityAnalyzer()
# sid.polarity_scores(text_retrieval[0])
# query_results['results'][0]

query = 'neighboring property'
# vec = TfidfVectorizer(strip_accents = 'unicode', analyzer = 'word', min_df = 0.35, max_df = .95)
vec = CountVectorizer(strip_accents = 'unicode', analyzer = 'word', min_df = .35, max_df = .95)
tfidf = vec.fit_transform(text_retrieval)
tokens = vec.get_feature_names()
features = pandas.DataFrame(data = tfidf.toarray(), index = range(len(text_retrieval)), columns = tokens)
tokenizer = vec.build_tokenizer()
inverted_index = dict()
for i in range(len(tokens)):
#     print(tokens[i])
    inverted_index.update({tokens[i] : i})
        
query_vector = np.zeros(len(tokens))
for token in tokenizer(query):
    idx = inverted_index.get(token)
    if idx != None:
        query_vector[idx] += 1
for row in features.iterrows():
#     print(np.array(row[1]))
    print(sum(sum(cosine_similarity(query_vector.reshape(-1, 1), np.array(row[1]).reshape(-1, 1)))))
# query_results['results'][3]

302.0
881.0
282.0
1021.0


In [ ]:
# ids, titles = [], []
# current = cases.json()
# i = 0
# while current['next'] != None and i < 100:
#     #print(i)
#     for case in current['results']:
#         ids.append(case['id'])
#         titles.append(case['name'])
#     current = requests.get(url = current['next']).json()
#     i += 1

In [49]:
utils.get_request_caselaw('https://api.case.law/v1/cases/?search="civil court"').json()

{'count': 13901,
 'next': 'https://api.case.law/v1/cases/?cursor=eyJwIjogWzEzLjE0ODEwNywgNjIyNDI1NjAwMDAwLCA4ODIwNzhdfQ%3D%3D&search=%22civil+court%22',
 'previous': None,
 'results': [{'id': 1589229,
   'url': 'https://api.case.law/v1/cases/1589229/',
   'name': 'FULTON COUNTY CIVIL COURT v. ELZEY',
   'name_abbreviation': 'Fulton County Civil Court v. Elzey',
   'decision_date': '1960-04-14',
   'docket_number': '38245',
   'first_page': '520',
   'last_page': '523',
   'citations': [{'cite': '101 Ga. App. 520', 'type': 'official'}],
   'volume': {'volume_number': '101',
    'url': 'https://api.case.law/v1/volumes/32044078445228/',
    'barcode': '32044078445228'},
   'reporter': {'full_name': 'Georgia Appeals Reports',
    'id': 519,
    'url': 'https://api.case.law/v1/reporters/519/'},
   'court': {'name_abbreviation': 'Ga. Ct. App.',
    'id': 8892,
    'url': 'https://api.case.law/v1/courts/ga-ct-app/',
    'name': 'Court of Appeals of Georgia',
    'slug': 'ga-ct-app'},
   'juri